<a href="https://colab.research.google.com/github/changsksu/IMSE_Data_Science/blob/main/Hypothesis_Testings_z_t_F_paired_t_CI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This nb demonstrates One-Sample and Two-Sample hypothesis tests via the use of direct codes, statsmodels weightstats, and scipy ttest functions.




In addition, codes to compute critical values for z and t tests, confidence intervals, and pair-t-test are also included.

Ref. https://k-state.instructure.com/courses/144373/modules/items/4455306
Ref. https://www.kite.com/python/docs/scipy.stats.ttest_1samp


In [1]:
import math
import statistics
import numpy as np
import scipy.stats
import pandas as pd
from statsmodels.stats import weightstats as stests
import scipy.stats as stats
from scipy.stats import norm
from scipy.stats import t
import matplotlib.pyplot as plt

In [2]:
import platform
print(platform.python_version())

3.10.12


In [ ]:
scipy.version

<module 'scipy.version' from '/usr/local/lib/python3.10/dist-packages/scipy/version.py'>

In [4]:
# make sure that you use the raw format in Github
# the data is under Dr. Chang's github changsksu folder KState_IMSE541
# use the URL once you find or store the data file (use the raw tab)
# the following input statement is needed to use pandas dataframe feature
import pandas as pd
data = pd.read_csv('https://raw.githubusercontent.com/changsksu/KState_IMSE541/main/brain_size.csv', sep=';', na_values=".")
# the foloowing statement only retrieves the first 5 obs
data.head()

,Unnamed: 0,Gender,FSIQ,VIQ,PIQ,Weight,Height,MRI_Count
0,1,Female,133,132,124,118.0,64.5,816932
1,2,Male,140,150,124,NaN,72.5,1001121
2,3,Male,139,123,150,143.0,73.3,1038437
3,4,Male,133,129,128,172.0,68.8,965353
4,5,Female,137,132,134,147.0,65.0,951545


# I. One-Sample Tests
**Ia Ztest when variance is known: Compute p-value via the scipy norm function**

In [5]:
viqmean=np.mean(data['VIQ'])
viqmean

112.35

In [6]:
# np.std
# ddof is the degrees of freedom, which is 1 for sample standard deviation and 0 for population standard deviation
viqstd=np.std(data['VIQ'], ddof=1)
viqstd

23.616107063199742

In [10]:
# assuming the variance of VIQ is known
# H0: mu = 110
# H1: mu > 110
# std=20 (known)
n=len(data['VIQ'])
pval=1-norm.cdf((viqmean-110)/(20/np.sqrt(n)))
pval

0.22869990000730867

Since pval is larger than 0.05, we fail to reject H0. Another way to cite this result is that reject H0 at (1-pval)*100% significant level

**Ib. t test when variance is unknown: Compute p-value via the scipy t function**

In [11]:
# assuming the variance of VIQ is unknown
# H0: mu = 110
# H1: mu > 110
# std should be estimated
n=len(data['VIQ'])
viqstd=np.std(data['VIQ'], ddof=1)
cdf = t.cdf((viqmean-110)/(viqstd/np.sqrt(n)), df=(n-1))
pval=1-cdf
pval

0.26639602500194526

In [14]:
#Two-side p-value
2*pval

0.5327920500038905

# Analysis:
if one sided test H1>110, since pval is larger than 0.05, we cannot reject H0
if H1<110, then pval=norm.cdf((viqmean-110)/(viqstd/np.sqrt(n)))
if two sided test and pval is smaller than 0.025, we would have reject H0. Or multiply the pval*2 and use 0.05 as the criterion.
Replace norm.cdf by t.cdf if standard deviation is estimated

**Ic. using scipy ttest_1samp function**

In [12]:
# one sample t test (default two sided test)
# there are two ways to run this one sample t test
stats.ttest_1samp(data['VIQ'], 110)
#scipy.stats.ttest_1samp(a=data['VIQ'], popmean=110)

TtestResult(statistic=0.6293461053092636, pvalue=0.5327920500038905, df=39)

Note that the default is two-side here. The method in 1b is one sided test

**Id. One Sample Test: using statsmodel's weightstats function**

ztest(x, y, value=110, alternative='two-sided')


*   x is the first sample
*   y is the second sample; set it at None for one sample test
*   alternative is the alternative hypothesis, which can be 'two-sided', 'larger', or 'smaller'; default alternative='two-sided'

Ref. https://www.statsmodels.org/dev/generated/statsmodels.stats.weightstats.ztest.html

In [15]:
# this function can also be used to test two samples!
# assuming the variance of VIQ is known or n > 30
# one-side H1 test is demonstrated here
ztest, propability_value = stests.ztest(data['VIQ'], x2=None, value=110, alternative="larger")
print(float(propability_value))
if propability_value<0.05:
  print("Null hyphothesis rejected")
else:
  print("Cannot reject the Null hyphothesis accepted ")

0.26456124668392544
Cannot reject the Null hyphothesis accepted 


In [16]:
# this function can also be used to test two samples!
# assuming the variance of VIQ is known or n > 30
# two-sided test H1 is demonstrated here
ztest, propability_value = stests.ztest(data['VIQ'], x2=None, value=110, alternative="two-sided")
print(float(propability_value))
if propability_value<0.05:
  print("Null hyphothesis rejected")
else:
  print("Cannot reject the Null hyphothesis accepted ")

0.5291224933678509
Cannot reject the Null hyphothesis accepted 


Please note that this result is slightly different from the "hand" calculation. The reason is that the "known" std is not provided when 1-sample z test is used.

In [19]:
# viqstd is not computed but z test (not t-test) is used
# this result is close to the t test result when n is large (i.e.  > 30)
n=len(data['VIQ'])
viqstd=np.std(data['VIQ'], ddof=1)
pval1 = norm.cdf((viqmean-110)/(viqstd/np.sqrt(n)))
pval2=1-pval1
pval2

0.26456124668392544

**Until we understand more of the Statmodel weightstats function, we should use hand computations or scipy ttest!**

# II. Two Sample Tests

**Case I. Pooled variances of two populations**

In [20]:
#using scipy
#two sample t-test 1: assume the variance is the same (default)
male_viq=data[data['Gender']=='Male']['VIQ']
female_viq=data[data['Gender']=='Female']['VIQ']
stats.ttest_ind(female_viq, male_viq, equal_var=True)

TtestResult(statistic=-0.7726161723275012, pvalue=0.44452876778583217, df=38.0)

**Case II. Reject equal variance assumptions**

In [21]:
#two sample t-test method 2: the variances are not the same
stats.ttest_ind(female_viq, male_viq, equal_var=False)

TtestResult(statistic=-0.7726161723275012, pvalue=0.44466074519419074, df=36.973150284539706)

Analysis: in both cases, the pvalues are large. We cannot reject H0.

# III. F test
**To judge Case I or Case II should be used**

In [ ]:
#F test for variances of two populations
# the following function is user defined
def f_test(x,y):
  x=np.array(x)
  y=np.array(y)
  f=np.var(x,ddof=1)/np.var(y, ddof=1) # cal F test statistic
  dfn=x.size-1
  dfd=y.size-1
  p=1-scipy.stats.f.cdf(f,dfn, dfd) #find p-value of F test statistics
  return f, p

f_test(male_viq, female_viq)

(1.3999574131873427, 0.23514586725149234)

The first output is the F statistic and the second is the p-value. Since the p-value is large, we cannot reject H0: var1=var2. Therefore, the variances of two populations should be pooled.

# **IV. Paired T-Test**

In [ ]:
#Paired t-Test method 1: using one sample t test
#Test if FISQ and PIQ are signficantly different
#The data for the pair (FISQ, PIQ) are obtained for the same subject
diff=data['FSIQ']-data['VIQ']
stats.ttest_1samp(diff, 0)


Ttest_1sampResult(statistic=0.8913911665594247, pvalue=0.37818604189634275)

The above p-value is large, which suggests H0 can not be rejected.

In [ ]:
#Paired t-Test method 2: using repeated measures test
#Test if FISQ and PIQ are signficantly different
#The data for the pair (FISQ, PIQ) are obtained for the same subject

stats.ttest_rel(data['FSIQ'],data['VIQ'])

Ttest_relResult(statistic=0.8913911665594247, pvalue=0.37818604189634275)

# **V. Critical Values of a t Distribution**

In [ ]:
# critcal values of a t distribution
# for two-sided 95% CI, the left critical value can be optained
x=data['VIQ']
df1=len(x) -1
stats.t.ppf(q=0.025, df=df1)


-2.0226909117347285

In [ ]:
# critcal values of a t distribution
# for two sided 95% CI, the right critical value can be optained

stats.t.ppf(q=0.975, df=df1)


2.022690911734728

In [ ]:
# critcal values of a t distribution
# for one sided 95% CI, the right critical value can be optained
# Ha u > u0

stats.t.ppf(q=0.95, df=df1)

1.6848751194973992

# **VI. Construct a Confidence Interval of a Sample**

In [22]:
# 95% confidence interval (variance estimated)
x=data['VIQ']
n=len(x)
df1=n-1
mean1=np.mean(x)
sigma1=np.std(x, ddof=1)/np.sqrt(n)
stats.t.interval(0.95, df=df1, loc=mean1, scale=sigma1)

(104.79720257664772, 119.90279742335227)

In [24]:
# hand calculation to verify the upper confidence interval limit

t0975=stats.t.ppf(q=0.975, df=df1)
UCL= mean1 + t0975 * sigma1
UCL

119.90279742335227

Your turn: can you compute the lower confidence interval limit?

In [25]:
# 95% confidence interval (variance known)
# known std=20
x=data['VIQ']
mean1=np.mean(x)
stats.norm.interval(0.95, loc=mean1, scale=20/np.sqrt(n))

(106.15204967695438, 118.54795032304561)

Your turn: can you constuct a 99% CI of x?